# 2. Exploratory Data Analysis (on all files of a certain taxi type, using spark)

In [18]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .getOrCreate()
)

In [4]:
sdf_yellow = spark.read.parquet('../data/raw/NYC_Taxi/yellow/')
sdf_yellow.show(1, vertical=True, truncate=100)

-RECORD 0------------------------------------
 VendorID              | 1                   
 tpep_pickup_datetime  | 2016-08-01 10:40:37 
 tpep_dropoff_datetime | 2016-08-01 10:41:37 
 passenger_count       | 1                   
 trip_distance         | 0.0                 
 RatecodeID            | 1                   
 store_and_fwd_flag    | N                   
 PULocationID          | 145                 
 DOLocationID          | 145                 
 payment_type          | 2                   
 fare_amount           | 2.5                 
 extra                 | 0.5                 
 mta_tax               | 0.5                 
 tip_amount            | 0.0                 
 tolls_amount          | 0.0                 
 improvement_surcharge | 0.3                 
 total_amount          | 3.8                 
 congestion_surcharge  | null                
 airport_fee           | null                
only showing top 1 row



In [5]:
# get count
sdf_yellow.count()

131131805

In [27]:
sdf = spark.read.parquet(f'../data/raw/NYC_Taxi/green/')
sdf = sdf.drop('congestion_surcharge')
sdf.show(1, vertical=True, truncate=100)

-RECORD 0------------------------------------
 VendorID              | 2                   
 lpep_pickup_datetime  | 2016-03-01 00:49:58 
 lpep_dropoff_datetime | 2016-03-01 01:06:33 
 store_and_fwd_flag    | N                   
 RatecodeID            | 1                   
 PULocationID          | 189                 
 DOLocationID          | 165                 
 passenger_count       | 1                   
 trip_distance         | 4.12                
 fare_amount           | 14.5                
 extra                 | 0.5                 
 mta_tax               | 0.5                 
 tip_amount            | 0.0                 
 tolls_amount          | 0.0                 
 ehail_fee             | null                
 improvement_surcharge | 0.3                 
 total_amount          | 15.8                
 payment_type          | 2                   
 trip_type             | 1.0                 
only showing top 1 row



In [11]:
# try to play with Ratecode - to see whether could do topic of predicting cancelled trips
sdf_yellow.filter((col('RatecodeID') == 3)).describe()

summary,VendorID,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
count,260258,260258,260258,260258,260258,260258,260258,260258,260258,260258,260258,260258,260258,260258,260258,0,0
mean,1.516402953991808,1.7160433108684459,16.35740826410718,3.0,null,159.14338079905323,32.17893782323694,1.3418223455186775,66.93041762405,0.20424079182964597,1.248760845007646...,10.010662419599011,13.03131634762424,0.29833549785224484,90.47944981519663,null,null
stddev,0.4998471499671104,1.3158939889983203,6.310732487315374,0.0,null,67.1703569690323,78.45447295807202,0.5799481447520934,667.921861863024,0.9707604969239015,0.007900803001161303,8.903531999999853,10.583100708610878,0.03144866145888842,668.4696733384926,null,null
min,1,0,0.0,3,N,1,1,1,-44.0,-1.0,0.0,-6.09,-17.12,-0.3,-45.3,null,null
max,3,9,258.8,3,Y,265,265,4,325478.79,406.26,0.5,238.0,946.0,0.6,325496.03,null,null


In [12]:
# get counts of number of ratecode = 3 (cancelled trips?)
sdf_yellow.filter((col('RatecodeID') == 3)).count()

260258

In [8]:
# trying to see how many invalid inputted rows in terms of negative total amount there
# was - could also point to cancelled trips
sdf_yellow.filter((col('total_amount') <0)).count()

55664